In [4]:
import json
import pandas as pd
import urllib.request
import urllib.parse

In [5]:
def latlon_from_address(address):
      
    url = 'http://loc.geopunt.be/v2/Location?'
    
    #strip spaces for more robustness
    values = {'q' : address.strip()}
    
    data = urllib.parse.urlencode(values)

    #get request is url + data
    #for POST request you need data.encode('utf8') first and use binary_data as a second arg instead of a '+'
    req = urllib.request.Request(url + data)
    
    req.add_header('Content-Type', 'text/json')
    response = urllib.request.urlopen(req)
    binary_response = response.read()
    decoded = binary_response.decode('utf8')
    
    response.close()

    jsonobj = json.loads(decoded)
    
    results = jsonobj['LocationResult']
    
    if len(results) == 0:
        return None
    
    location = results[0]['Location']

    latkeyID = 'Lat_WGS84'
    lonkeyID = 'Lon_WGS84'
    
    return (location[latkeyID], location[lonkeyID])
        

In [6]:
all_data = pd.read_csv('EVA_restoswithlocations.csv')
all_data.head(n=3)

,city,name,street,tags,zipcode,full_address,latlon,missing
0,Neerijse,Biesbemd,Kamstraat 33,"['Approved by EVA', 'EVA voordeel', 'Veganvrie...",3040,Kamstraat 33 3040 Neerijse,"(50.8127340562564, 4.622892881244533)",False
1,Antwerpen,Falafel Shami,Hoogstraat 47,"['Snack', '100% vegetarisch', 'Approved by EVA...",2000,Hoogstraat 47 2000 Antwerpen,"(51.21912277399896, 4.39791278664577)",False
2,Antwerpen,Falafel Tof,Hoogstraat 32,"['Eethuis', 'Snack', '100% vegetarisch', 'Appr...",2000,Hoogstraat 32 2000 Antwerpen,"(51.219967595445304, 4.398256839416387)",False


In [7]:
df_notfound = all_data[all_data['missing']]
print(df_notfound.shape)
print(all_data.shape)



(91, 8)
(735, 8)


In [8]:
def overwriteAddress(full_address, index, dataframe):
    
    parts = full_address.split(' ')
    
    city = parts[-1]
    zipcode = int(parts[-2])
    street = " ".join(parts[:len(parts)-2])
    latlon = latlon_from_address(full_address)
    
    dataframe.set_value(index, 'street',  street)
    dataframe.set_value(index, 'city',    city)
    dataframe.set_value(index, 'zipcode', zipcode)
    dataframe.set_value(index, 'full_address', full_address)
    dataframe.set_value(index, 'latlon', latlon)
    dataframe.set_value(index, 'missing', False)


def overwriteAddress2(street,zipcode,city, index, dataframe):
    
    full_address = " ".join([street,zipcode,city])
    latlon = latlon_from_address(full_address)
    
    dataframe.set_value(index, 'street',  street)
    dataframe.set_value(index, 'city',    city)
    dataframe.set_value(index, 'zipcode', zipcode)
    dataframe.set_value(index, 'full_address', full_address)
    dataframe.set_value(index, 'latlon', latlon)
    dataframe.set_value(index, 'missing', False)




In [9]:
df_notfound[0:10]

,city,name,street,tags,zipcode,full_address,latlon,missing
7,Baal,The Vexican,C. Huysmansstraat 126,"['Cateraar', 'Snack', '100% vegetarisch', 'App...",3128,C. Huysmansstraat 126 3128 Baal,NaN,True
16,Antwerpen,'t Koekebakske,Leeuwestraat 23,"['Eethuis', 'EVA voordeel']",2000,Leeuwestraat 23 2000 Antwerpen,NaN,True
24,Ieper,Agadir,Rijselsestraat 42,"['Eethuis', 'Snack']",8900,Rijselsestraat 42 8900 Ieper,NaN,True
32,Wetteren,Amuz&Vous,V. Van Sandelaan 33,['Cateraar'],9230,V. Van Sandelaan 33 9230 Wetteren,NaN,True
46,Waregem,BAOBAB Catering,"Mirakelstraat 104A, 8790 Waregem",['Cateraar'],8790,"Mirakelstraat 104A, 8790 Waregem 8790 Waregem",NaN,True
62,Sint-Niklaas,Biko,Sacramentstraat 3,"['Eethuis', 'Snack']",9100,Sacramentstraat 3 9100 Sint-Niklaas,NaN,True
68,Aalst,Black and White,Koostraat 107,['Eethuis'],9300,Koostraat 107 9300 Aalst,NaN,True
81,Oostende,Brasserie Buyl,A. Buylstraat 44B,['Eethuis'],8400,A. Buylstraat 44B 8400 Oostende,NaN,True
91,Gent,Café De Kleine Kunst,Lousbergkaai 99,"['Eethuis', 'EVA voordeel']",9000,Lousbergkaai 99 9000 Gent,NaN,True
98,Gent,Café Parti,Maria-Hendrikaplein 65a,"['Eethuis', 'Veganvriendelijk']",9000,Maria-Hendrikaplein 65a 9000 Gent,NaN,True


In [10]:
overwriteAddress('Camille Huysmansstraat 126 3128 Tremelo',7,df_notfound)
overwriteAddress('Leeuwenstraat 23 2000 Antwerpen',16,df_notfound)
overwriteAddress('Rijselstraat 42 8900 Ieper',24,df_notfound)
overwriteAddress('Victor Van Sandelaan 33 9230 Wetteren',32,df_notfound)
overwriteAddress('Mirakelstraat 104A 8790 Waregem',46,df_notfound)
overwriteAddress('Sacramentsstraat 3 9100 Sint-Niklaas',62,df_notfound)
overwriteAddress('Koolstraat 107 9300 Aalst',68,df_notfound)
overwriteAddress('Adolf Buylstraat 44B 8400 Oostende',81,df_notfound)
overwriteAddress('Ferdinand Lousbergkaai 99 9000 Gent',91,df_notfound)
overwriteAddress('Koningin Maria-Hendrikaplein 65a 9000 Gent',98,df_notfound)

df_notfound[0:10]

,city,name,street,tags,zipcode,full_address,latlon,missing
7,Tremelo,The Vexican,Camille Huysmansstraat 126,"['Cateraar', 'Snack', '100% vegetarisch', 'App...",3128,Camille Huysmansstraat 126 3128 Tremelo,"(50.99981171485267, 4.749558921517834)",False
16,Antwerpen,'t Koekebakske,Leeuwenstraat 23,"['Eethuis', 'EVA voordeel']",2000,Leeuwenstraat 23 2000 Antwerpen,"(51.218385351929605, 4.399357877940981)",False
24,Ieper,Agadir,Rijselstraat 42,"['Eethuis', 'Snack']",8900,Rijselstraat 42 8900 Ieper,"(50.849543772226255, 2.8863381665928447)",False
32,Wetteren,Amuz&Vous,Victor Van Sandelaan 33,['Cateraar'],9230,Victor Van Sandelaan 33 9230 Wetteren,"(51.00217297024971, 3.8875914088732486)",False
46,Waregem,BAOBAB Catering,Mirakelstraat 104A,['Cateraar'],8790,Mirakelstraat 104A 8790 Waregem,"(50.870729494598294, 3.397632962878593)",False
62,Sint-Niklaas,Biko,Sacramentsstraat 3,"['Eethuis', 'Snack']",9100,Sacramentsstraat 3 9100 Sint-Niklaas,"(51.16461233026816, 4.142836540827675)",False
68,Aalst,Black and White,Koolstraat 107,['Eethuis'],9300,Koolstraat 107 9300 Aalst,"(50.939983118445085, 4.02898384827834)",False
81,Oostende,Brasserie Buyl,Adolf Buylstraat 44B,['Eethuis'],8400,Adolf Buylstraat 44B 8400 Oostende,"(51.23115235995707, 2.9150036519826585)",False
91,Gent,Café De Kleine Kunst,Ferdinand Lousbergkaai 99,"['Eethuis', 'EVA voordeel']",9000,Ferdinand Lousbergkaai 99 9000 Gent,"(51.04753532194943, 3.7392711886277534)",False
98,Gent,Café Parti,Koningin Maria-Hendrikaplein 65a,"['Eethuis', 'Veganvriendelijk']",9000,Koningin Maria-Hendrikaplein 65a 9000 Gent,"(51.036379827654144, 3.7161672129454875)",False


In [11]:
df_notfound[10:20]

,city,name,street,tags,zipcode,full_address,latlon,missing
100,Brussel,Cafeabc,Phillippe de Champagnestraat 23,"['Eethuis', 'Snack']",1000,Phillippe de Champagnestraat 23 1000 Brussel,NaN,True
124,Drogenbos,Cook-Om,Rue Marie Collart,"['Eethuis', 'EVA voordeel', 'Veganvriendelijk']",1620,Rue Marie Collart 1620 Drogenbos,NaN,True
128,Elsene,Crêperie au p’tit Breton,Amerikaansestraat 117,['Snack'],1050,Amerikaansestraat 117 1050 Elsene,NaN,True
132,Mechelen,d'Afspraak,Keizeerstraat 23,['Eethuis'],2800,Keizeerstraat 23 2800 Mechelen,NaN,True
139,Wellen,De Bottelarij,Ulbeekstraat 21,['Eethuis'],3830,Ulbeekstraat 21 3830 Wellen,NaN,True
140,Brugge,De Bottelier,St.Jacobsstraat 63,['Eethuis'],8000,St.Jacobsstraat 63 8000 Brugge,NaN,True
147,Leuven,De Dijlemolens,Zwarte zustersstraat 16/4,['Eethuis'],3000,Zwarte zustersstraat 16/4 3000 Leuven,NaN,True
153,Sint-Pieters-Rode,De Gelaarsde Kat,Gobbelsrode 68a,['Eethuis'],3320,Gobbelsrode 68a 3320 Sint-Pieters-Rode,NaN,True
161,nvriendelijk,De Grote post,Eethuis,[],Vega,Eethuis Vega nvriendelijk,NaN,True
163,Brugge - Sint Kruis,De Jonkman,Maaltsesteenweg 438,"['Gastronomisch', 'Veganvriendelijk']",8310,Maaltsesteenweg 438 8310 Brugge - Sint Kruis,NaN,True


In [12]:
# spelfout, fr/nl, fr/nl, spelfout, postcode
overwriteAddress('Philippe de Champagnestraat 23 1000 Brussel',100,df_notfound)
overwriteAddress('Marie Collartstraat 1620 Drogenbos',124,df_notfound)
overwriteAddress('Rue Américaine 117 1050 Elsene',128,df_notfound)
overwriteAddress('Keizerstraat 23 2800 Mechelen',132,df_notfound)
overwriteAddress('Ulbeekstraat 21 3832 Wellen',139,df_notfound)

# afkorting+spelfouten, spelfout, stad,  bad record, contractie
overwriteAddress('Sint Jakobstraat 63 8000 Brugge',140,df_notfound)
overwriteAddress('Zwartzustersstraat 16 3000 Leuven',147,df_notfound)
overwriteAddress('Gobbelsrode 68a 3220 Holsbeek',153,df_notfound)
#bad record
overwriteAddress2('Maalse Steenweg 438', '8310', 'Brugge-Sint Kruis',163,df_notfound)

df_notfound[10:20]

,city,name,street,tags,zipcode,full_address,latlon,missing
100,Brussel,Cafeabc,Philippe de Champagnestraat 23,"['Eethuis', 'Snack']",1000,Philippe de Champagnestraat 23 1000 Brussel,"(50.84314226758593, 4.347341194697616)",False
124,Drogenbos,Cook-Om,Marie Collartstraat,"['Eethuis', 'EVA voordeel', 'Veganvriendelijk']",1620,Marie Collartstraat 1620 Drogenbos,"(50.78156305645173, 4.31661255727227)",False
128,Elsene,Crêperie au p’tit Breton,Rue Américaine 117,['Snack'],1050,Rue Américaine 117 1050 Elsene,"(50.82249491537016, 4.361301054768735)",False
132,Mechelen,d'Afspraak,Keizerstraat 23,['Eethuis'],2800,Keizerstraat 23 2800 Mechelen,"(51.0290618159853, 4.488280080616929)",False
139,Wellen,De Bottelarij,Ulbeekstraat 21,['Eethuis'],3832,Ulbeekstraat 21 3832 Wellen,"(50.84097654956267, 5.308105034230186)",False
140,Brugge,De Bottelier,Sint Jakobstraat 63,['Eethuis'],8000,Sint Jakobstraat 63 8000 Brugge,"(51.20801546511949, 3.2228642331128525)",False
147,Leuven,De Dijlemolens,Zwartzustersstraat 16,['Eethuis'],3000,Zwartzustersstraat 16 3000 Leuven,"(50.87305970607464, 4.696587161773074)",False
153,Holsbeek,De Gelaarsde Kat,Gobbelsrode 68a,['Eethuis'],3220,Gobbelsrode 68a 3220 Holsbeek,"(50.915747648372886, 4.807520754675542)",False
161,nvriendelijk,De Grote post,Eethuis,[],Vega,Eethuis Vega nvriendelijk,NaN,True
163,Brugge-Sint Kruis,De Jonkman,Maalse Steenweg 438,"['Gastronomisch', 'Veganvriendelijk']",8310,Maalse Steenweg 438 8310 Brugge-Sint Kruis,"(51.20768556028524, 3.2782982449692675)",False


In [13]:
df_notfound[20:30]

,city,name,street,tags,zipcode,full_address,latlon,missing
167,Rumbeke,De Kleine Prins,Rijstelstraat 5,['Eethuis'],8800,Rijstelstraat 5 8800 Rumbeke,NaN,True
172,"Vorst, Kempen",De Kruimel,Dikstraat 3,['Eethuis'],2430,"Dikstraat 3 2430 Vorst, Kempen",NaN,True
189,Kortrijk,De Promenade,Veemlarkt 30,"['Eethuis', 'Snack']",8500,Veemlarkt 30 8500 Kortrijk,NaN,True
192,Brugge,De Republiek,St. Jacobsstraat 36,"['Eethuis', 'Snack']",8000,St. Jacobsstraat 36 8000 Brugge,NaN,True
194,Zoutenaaie,De Reygaerd,Reygaerddijkstraat 22,['Eethuis'],8630,Reygaerddijkstraat 22 8630 Zoutenaaie,NaN,True
197,Stokrooie,De Stroobander,St. Jozefsplein 13,['Eethuis'],3511,St. Jozefsplein 13 3511 Stokrooie,NaN,True
201,Gent,De Walrus,"Coupure Links 497,",['Eethuis'],9000,"Coupure Links 497, 9000 Gent",NaN,True
224,Hasselt,Double Dragons,Hasseltweg 214,['Eethuis'],3500,Hasseltweg 214 3500 Hasselt,NaN,True
232,Oostende,El Mariachi,Kapucijnestraat 44,['Eethuis'],8400,Kapucijnestraat 44 8400 Oostende,NaN,True
239,Sint-Maria-Horebeke,Elim,Dorpstraat 30,['Eethuis'],9667,Dorpstraat 30 9667 Sint-Maria-Horebeke,NaN,True


In [15]:
overwriteAddress('Rijselstraat 5 8800 Rumbeke',167,df_notfound)
overwriteAddress('Dikstraat 3 2430 Laakdal',172,df_notfound)
overwriteAddress('Veemarkt 30 8500 Kortrijk',189,df_notfound)
overwriteAddress('Sint Jacobsstraat 36 8000 Brugge',192,df_notfound)
overwriteAddress('Reygaerdijkstraat 22 8630 Zoutenaaie',194,df_notfound)

overwriteAddress('Sint Jozefsplein 13 3511 Stokrooie',197,df_notfound)
overwriteAddress('Coupure Links 497 9000 Gent',201,df_notfound)
overwriteAddress('Genkersteenweg 214 3500 Hasselt',224,df_notfound)
overwriteAddress('Kapucijnenstraat 44 8400 Oostende',232,df_notfound)
overwriteAddress('Dorpsstraat 30 9667 Sint-Maria-Horebeke',239,df_notfound)
df_notfound[20:30]

,city,name,street,tags,zipcode,full_address,latlon,missing
167,Rumbeke,De Kleine Prins,Rijselstraat 5,['Eethuis'],8800,Rijselstraat 5 8800 Rumbeke,"(50.940853119308244, 3.123873800952088)",False
172,Laakdal,De Kruimel,Dikstraat 3,['Eethuis'],2430,Dikstraat 3 2430 Laakdal,"(51.08777822444371, 5.0689052810212445)",False
189,Kortrijk,De Promenade,Veemarkt 30,"['Eethuis', 'Snack']",8500,Veemarkt 30 8500 Kortrijk,"(50.82624906812174, 3.2734970141452155)",False
192,Brugge,De Republiek,Sint Jacobsstraat 36,"['Eethuis', 'Snack']",8000,Sint Jacobsstraat 36 8000 Brugge,"(51.20801546511949, 3.2228642331128525)",False
194,Zoutenaaie,De Reygaerd,Reygaerdijkstraat 22,['Eethuis'],8630,Reygaerdijkstraat 22 8630 Zoutenaaie,"(51.06465342677114, 2.7307744620794177)",False
197,Stokrooie,De Stroobander,Sint Jozefsplein 13,['Eethuis'],3511,Sint Jozefsplein 13 3511 Stokrooie,"(50.966124160569926, 5.279991265198184)",False
201,Gent,De Walrus,Coupure Links 497,['Eethuis'],9000,Coupure Links 497 9000 Gent,"(51.055719748430406, 3.7087946627413633)",False
224,Hasselt,Double Dragons,Genkersteenweg 214,['Eethuis'],3500,Genkersteenweg 214 3500 Hasselt,"(50.94450048610327, 5.361322843381503)",False
232,Oostende,El Mariachi,Kapucijnenstraat 44,['Eethuis'],8400,Kapucijnenstraat 44 8400 Oostende,"(51.233991155864814, 2.9177648388281234)",False
239,Sint-Maria-Horebeke,Elim,Dorpsstraat 30,['Eethuis'],9667,Dorpsstraat 30 9667 Sint-Maria-Horebeke,"(50.83845885800066, 3.688273556090574)",False


In [16]:
df_notfound[30:40]

,city,name,street,tags,zipcode,full_address,latlon,missing
240,Mechelen,Ellis Gourmet burger,Ijzerleen 10,['Eethuis'],2800,Ijzerleen 10 2800 Mechelen,NaN,True
243,Les Bulles - Nederland,Enig Alternatief,Wielakkerstraat 4,['Eethuis'],6811,Wielakkerstraat 4 6811 Les Bulles - Nederland,NaN,True
247,Elsene,Exki,Elsensesteenweg 12,['Snack'],1050,Elsensesteenweg 12 1050 Elsene,NaN,True
254,Gent,Fabula Rasa,Fr. Lousbergkaai 134,['Eethuis'],9000,Fr. Lousbergkaai 134 9000 Gent,NaN,True
259,Gent,Falafel Punt,Rooiegemlaan 389,"['Eethuis', 'EVA voordeel', 'Veganvriendelijk']",9000,Rooiegemlaan 389 9000 Gent,NaN,True
269,Gent,Foley's Irish Pub,Recolettenlei 10,['Eethuis'],9000,Recolettenlei 10 9000 Gent,NaN,True
287,Leuven,Frituur Tervuursepoort,Tervuursepoort 1,[],3000,Tervuursepoort 1 3000 Leuven,NaN,True
292,Vucht,Gastronomia Cellini,Maasmechelen Village,['Eethuis'],3630,Maasmechelen Village 3630 Vucht,NaN,True
309,Brussel,Happy Buddha,E. Jacqmainlaan 7,"['Eethuis', 'EVA voordeel']",1000,E. Jacqmainlaan 7 1000 Brussel,NaN,True
317,Kessel - Lo,Het geheim,B.A. de Becker-Remyplein 19,['Eethuis'],3010,B.A. de Becker-Remyplein 19 3010 Kessel - Lo,NaN,True


In [18]:
overwriteAddress('IJzerenleen 10 2800 Mechelen',240,df_notfound)
#print(latlon_from_address('')) #NIET GEVONDEN
overwriteAddress("Chaussée D'Ixelles 12 1050 Ixelles",247,df_notfound)
overwriteAddress('Lousbergskaai 134 9000 Gent',254,df_notfound)
overwriteAddress('Rooigemlaan 389 9000 Gent',259,df_notfound)

overwriteAddress('Recollettenlei 10 9000 Gent',269,df_notfound)
overwriteAddress('Tervuursevest 3000 Leuven',287,df_notfound)
#print(latlon_from_address('')) #FAILLIET
overwriteAddress('Emile Jacqmainlaan 7 1000 Brussel',309,df_notfound)
overwriteAddress('Baron August de Becker Remyplein 19 3010 Leuven',317,df_notfound)

df_notfound[30:40]

,city,name,street,tags,zipcode,full_address,latlon,missing
240,Mechelen,Ellis Gourmet burger,IJzerenleen 10,['Eethuis'],2800,IJzerenleen 10 2800 Mechelen,"(51.02739024945941, 4.478711258745668)",False
243,Les Bulles - Nederland,Enig Alternatief,Wielakkerstraat 4,['Eethuis'],6811,Wielakkerstraat 4 6811 Les Bulles - Nederland,NaN,True
247,Ixelles,Exki,Chaussée D'Ixelles 12,['Snack'],1050,Chaussée D'Ixelles 12 1050 Ixelles,"(50.83294097077088, 4.366778892054664)",False
254,Gent,Fabula Rasa,Lousbergskaai 134,['Eethuis'],9000,Lousbergskaai 134 9000 Gent,"(51.04753532194943, 3.7392711886277534)",False
259,Gent,Falafel Punt,Rooigemlaan 389,"['Eethuis', 'EVA voordeel', 'Veganvriendelijk']",9000,Rooigemlaan 389 9000 Gent,"(51.06112430697188, 3.6932695393981017)",False
269,Gent,Foley's Irish Pub,Recollettenlei 10,['Eethuis'],9000,Recollettenlei 10 9000 Gent,"(51.05000804846928, 3.719499990341022)",False
287,Leuven,Frituur Tervuursepoort,Tervuursevest,[],3000,Tervuursevest 3000 Leuven,"(50.87141275848566, 4.690779214197726)",False
292,Vucht,Gastronomia Cellini,Maasmechelen Village,['Eethuis'],3630,Maasmechelen Village 3630 Vucht,NaN,True
309,Brussel,Happy Buddha,Emile Jacqmainlaan 7,"['Eethuis', 'EVA voordeel']",1000,Emile Jacqmainlaan 7 1000 Brussel,"(50.85240239069649, 4.35351457245943)",False
317,Leuven,Het geheim,Baron August de Becker Remyplein 19,['Eethuis'],3010,Baron August de Becker Remyplein 19 3010 Leuven,"(50.88498605717642, 4.719266161742309)",False


In [19]:
df_notfound[40:50]

,city,name,street,tags,zipcode,full_address,latlon,missing
321,Sint-Niklaas,Het Laatste Avondmaal,Sacramentstraat 5,[],9100,Sacramentstraat 5 9100 Sint-Niklaas,NaN,True
322,Nieuwkerke,Het Labyrint,Dries 29,['Eethuis'],8950,Dries 29 8950 Nieuwkerke,NaN,True
337,Manhay,Hotel les Sources,Route du Crahay 28,"['Eethuis', 'EVA voordeel']",6960,Route du Crahay 28 6960 Manhay,NaN,True
354,Heuvelland ( Dranouter),In de Wulf,Wulvestraat 1,"['Gastronomisch', 'Veganvriendelijk']",8950,Wulvestraat 1 8950 Heuvelland ( Dranouter),NaN,True
358,Hasselt,IZIBILIBOCO,Dorpstraat 36,"['Eethuis', '100% vegetarisch', 'EVA voordeel']",3500,Dorpstraat 36 3500 Hasselt,NaN,True
384,Blankenberge,Klein Begin,Visserstraat 23,['Eethuis'],8370,Visserstraat 23 8370 Blankenberge,NaN,True
396,Antwerpen,Krua Thai,Pelgrimstrtaat 13,"['Eethuis', 'Veganvriendelijk']",2000,Pelgrimstrtaat 13 2000 Antwerpen,NaN,True
411,La-Roche-en-Ardenne,La Nature Gourmande,Rue Pumalet 9,['Eethuis'],6980,Rue Pumalet 9 6980 La-Roche-en-Ardenne,NaN,True
414,Elsene,La Porte des Indes,Louisalaan 455,['Eethuis'],1050,Louisalaan 455 1050 Elsene,NaN,True
419,Leuven,La Stanza,Wandelingenstraat 8,['Eethuis'],3000,Wandelingenstraat 8 3000 Leuven,NaN,True


In [20]:
overwriteAddress('Sacramentsstraat 9100 Sint-Niklaas',321,df_notfound)
overwriteAddress('Dries 29 8956 Nieuwkerke',322,df_notfound)

#(latlon_from_address('')) # Wallonia 50.262275, 5.656895
df_notfound.set_value(337, 'latlon', (50.262275, 5.656895))
df_notfound.set_value(337, 'missing', False)
    
    
overwriteAddress('Wulvestraat 1 8951 Heuvelland',354,df_notfound)
overwriteAddress('Dorpsstraat 36 3500 Hasselt',358,df_notfound)

overwriteAddress('Vissersstraat 23 8370 Blankenberge',384,df_notfound)
overwriteAddress('Pelgrimstraat 13 2000 Antwerpen',396,df_notfound)

#('Rue du purnalet 9 6980 La Roche-en-Ardenne')) #50.181260, 5.575643
df_notfound.set_value(411, 'latlon', (50.181260, 5.575643))
df_notfound.set_value(411, 'missing', False)    


overwriteAddress('Louizalaan 455 1050 Elsene',414,df_notfound)
overwriteAddress('Wandelingstraat 8 3000 Leuven',419,df_notfound)

df_notfound[40:50]

,city,name,street,tags,zipcode,full_address,latlon,missing
321,Sint-Niklaas,Het Laatste Avondmaal,Sacramentsstraat,[],9100,Sacramentsstraat 9100 Sint-Niklaas,"(51.1645671421227, 4.142708094155377)",False
322,Nieuwkerke,Het Labyrint,Dries 29,['Eethuis'],8956,Dries 29 8956 Nieuwkerke,"(50.7830491845225, 2.8268628247046723)",False
337,Manhay,Hotel les Sources,Route du Crahay 28,"['Eethuis', 'EVA voordeel']",6960,Route du Crahay 28 6960 Manhay,"(50.262275, 5.656895)",False
354,Heuvelland,In de Wulf,Wulvestraat 1,"['Gastronomisch', 'Veganvriendelijk']",8951,Wulvestraat 1 8951 Heuvelland,"(50.748353613531826, 2.792665289404845)",False
358,Hasselt,IZIBILIBOCO,Dorpsstraat 36,"['Eethuis', '100% vegetarisch', 'EVA voordeel']",3500,Dorpsstraat 36 3500 Hasselt,"(50.932453813844965, 5.335460073350613)",False
384,Blankenberge,Klein Begin,Vissersstraat 23,['Eethuis'],8370,Vissersstraat 23 8370 Blankenberge,"(51.31494640794891, 3.126780840833021)",False
396,Antwerpen,Krua Thai,Pelgrimstraat 13,"['Eethuis', 'Veganvriendelijk']",2000,Pelgrimstraat 13 2000 Antwerpen,"(51.21963466612017, 4.399630661487253)",False
411,La-Roche-en-Ardenne,La Nature Gourmande,Rue Pumalet 9,['Eethuis'],6980,Rue Pumalet 9 6980 La-Roche-en-Ardenne,"(50.18126, 5.575643)",False
414,Elsene,La Porte des Indes,Louizalaan 455,['Eethuis'],1050,Louizalaan 455 1050 Elsene,"(50.81778453806542, 4.370880391068708)",False
419,Leuven,La Stanza,Wandelingstraat 8,['Eethuis'],3000,Wandelingstraat 8 3000 Leuven,"(50.88101844104403, 4.698717461992025)",False


In [21]:
df_notfound[50:60]

,city,name,street,tags,zipcode,full_address,latlon,missing
424,Heldergem,Labaij Kris,Stationstraat 220,['Snack'],9450,Stationstraat 220 9450 Heldergem,NaN,True
425,Mechelen,Lan Na Thai,F. de Merodestraat 59,"['Eethuis', 'Veganvriendelijk']",2800,F. de Merodestraat 59 2800 Mechelen,NaN,True
428,Brussel,Le Bar à Tapas,Borgvalstraat 11,['Eethuis'],1000,Borgvalstraat 11 1000 Brussel,NaN,True
429,Brussel,Le Bar à Tapas,Borgvalstraat 11,['Eethuis'],1000,Borgvalstraat 11 1000 Brussel,NaN,True
430,Etterbeek,Le Bol de Riz,Waversesteenweg 335,['Eethuis'],1040,Waversesteenweg 335 1040 Etterbeek,NaN,True
432,Elsene,Le Grenier d'Elvire,Boondaalsesteenweg 339a,['Eethuis'],1050,Boondaalsesteenweg 339a 1050 Elsene,NaN,True
449,Elsene,Les Jardins de Bagatelle,Herderstraat 17,['Eethuis'],1050,Herderstraat 17 1050 Elsene,NaN,True
452,raar,Les Odettes,2000Regio Antwerpen,[],Cate,2000Regio Antwerpen Cate raar,NaN,True
457,plantaardig,Little Green Truck,Cateraar,"['100% vegetarisch', 'Veganvriendelijk']",100%,Cateraar 100% plantaardig,NaN,True
459,Antwerpen,Loes & Krikke bvba,Colonia Congresstraat 42/101,['Cateraar'],2060,Colonia Congresstraat 42/101 2060 Antwerpen,NaN,True


In [22]:
overwriteAddress('Stationsstraat 220 9450 Heldergem',424,df_notfound)
overwriteAddress('Frederik de Merodestraat 59 2800 Mechelen',425,df_notfound)
overwriteAddress('Rue Borgvalstraat 11 1000 Brussel',428,df_notfound)
#latlon_from_address('')) #DUPLICATE
overwriteAddress('Chaussée de Wavre 335 1040 Etterbeek',430,df_notfound)

overwriteAddress('Chaussée de Boondael 339a 1050 Ixelles',432,df_notfound)
overwriteAddress('Herdersstraat 17 1050 Elsene',449,df_notfound)
overwriteAddress('Dambruggestraat 36/1 2060 Antwerpen',452,df_notfound)
#('')) #eerder een verhuisservice!?
overwriteAddress('Congresstraat 2060 Laar',459,df_notfound)
df_notfound[50:60]

,city,name,street,tags,zipcode,full_address,latlon,missing
424,Heldergem,Labaij Kris,Stationsstraat 220,['Snack'],9450,Stationsstraat 220 9450 Heldergem,"(50.90644752252776, 4.024136175932285)",False
425,Mechelen,Lan Na Thai,Frederik de Merodestraat 59,"['Eethuis', 'Veganvriendelijk']",2800,Frederik de Merodestraat 59 2800 Mechelen,"(51.03057816302183, 4.482054488972484)",False
428,Brussel,Le Bar à Tapas,Rue Borgvalstraat 11,['Eethuis'],1000,Rue Borgvalstraat 11 1000 Brussel,"(50.847862473722664, 4.351854614231369)",False
429,Brussel,Le Bar à Tapas,Borgvalstraat 11,['Eethuis'],1000,Borgvalstraat 11 1000 Brussel,NaN,True
430,Etterbeek,Le Bol de Riz,Chaussée de Wavre 335,['Eethuis'],1040,Chaussée de Wavre 335 1040 Etterbeek,"(50.8307725515541, 4.389419219700109)",False
432,Ixelles,Le Grenier d'Elvire,Chaussée de Boondael 339a,['Eethuis'],1050,Chaussée de Boondael 339a 1050 Ixelles,"(50.81858364363958, 4.38338257537767)",False
449,Elsene,Les Jardins de Bagatelle,Herdersstraat 17,['Eethuis'],1050,Herdersstraat 17 1050 Elsene,"(50.836132098538194, 4.361810154666128)",False
452,Antwerpen,Les Odettes,Dambruggestraat 36/1,[],2060,Dambruggestraat 36/1 2060 Antwerpen,"(51.22430063798104, 4.421778083510996)",False
457,plantaardig,Little Green Truck,Cateraar,"['100% vegetarisch', 'Veganvriendelijk']",100%,Cateraar 100% plantaardig,NaN,True
459,Laar,Loes & Krikke bvba,Congresstraat,['Cateraar'],2060,Congresstraat 2060 Laar,"(51.21867154334976, 4.426752383324471)",False


In [23]:
df_notfound[60:70]

,city,name,street,tags,zipcode,full_address,latlon,missing
462,Brugge,Lotus,Wapenmakerstraat 5,['Eethuis'],8000,Wapenmakerstraat 5 8000 Brugge,NaN,True
497,Sint-Niklaas,Moon Eat,Richard Van Britsomstraat 18,[],9100,Richard Van Britsomstraat 18 9100 Sint-Niklaas,NaN,True
498,sel,Moonfood,Koloniënstraat 58,"['Eethuis', 'Snack']",Brus,Koloniënstraat 58 Brus sel,NaN,True
511,vegetarisch,Natuurfrituur,Cateraar,['Veganvriendelijk'],100%,Cateraar 100% vegetarisch,NaN,True
523,Elsene,O Liban,Vleurgatsesteenweg 324,"['Eethuis', 'Veganvriendelijk']",1050,Vleurgatsesteenweg 324 1050 Elsene,NaN,True
536,Sint-Gillis,Ozfair,J.Volderslaan,['Eethuis'],1060,J.Volderslaan 1060 Sint-Gillis,NaN,True
544,Brugge,Passage,Dweerstraat 26,['Eethuis'],8000,Dweerstraat 26 8000 Brugge,NaN,True
546,Brugge,Pasta Presto,St. Amandstraat 17,['Eethuis'],8000,St. Amandstraat 17 8000 Brugge,NaN,True
556,Gent,Petrus,Sint-Amandsstraat 15,"['Eethuis', 'Snack', 'Veganvriendelijk']",9000,Sint-Amandsstraat 15 9000 Gent,NaN,True
563,Hasselt,Pocomatto,Dorpstraat 42,['Eethuis'],3500,Dorpstraat 42 3500 Hasselt,NaN,True


In [24]:
overwriteAddress('Wapenmakersstraat 5 8000 Brugge',462,df_notfound)
overwriteAddress('Rich. van Britsomstraat 18, 9100 Sint-Niklaas',497,df_notfound)
overwriteAddress('Koloniënstraat 58 Brussel',498,df_notfound)
#print(latlon_from_address('')) #BAD RECORD
overwriteAddress('Chaussée de Vleurgat 324 1050 Ixelles',523,df_notfound)

overwriteAddress('Jean Volderslaan 1060 Sint-Gillis',536,df_notfound)
overwriteAddress('Dweersstraat 26 8000 Brugge',544,df_notfound)
overwriteAddress('Sint-Amandstraat 17 9000 Gent',546,df_notfound)
overwriteAddress('Sint-Amandstraat 15 9000 Gent',556,df_notfound)
overwriteAddress('Dorpsstraat 42 3500 Hasselt',563,df_notfound)

df_notfound[60:70]

,city,name,street,tags,zipcode,full_address,latlon,missing
462,Brugge,Lotus,Wapenmakersstraat 5,['Eethuis'],8000,Wapenmakersstraat 5 8000 Brugge,"(51.209812420194034, 3.226483870185225)",False
497,Sint-Niklaas,Moon Eat,"Rich. van Britsomstraat 18,",[],9100,"Rich. van Britsomstraat 18, 9100 Sint-Niklaas","(51.16850816904729, 4.140115558305267)",False
498,Brussel,Moonfood,Koloniënstraat,"['Eethuis', 'Snack']",58,Koloniënstraat 58 Brussel,"(50.84721615790801, 4.361368496059801)",False
511,vegetarisch,Natuurfrituur,Cateraar,['Veganvriendelijk'],100%,Cateraar 100% vegetarisch,NaN,True
523,Ixelles,O Liban,Chaussée de Vleurgat 324,"['Eethuis', 'Veganvriendelijk']",1050,Chaussée de Vleurgat 324 1050 Ixelles,"(50.82243218147768, 4.368737676178942)",False
536,Sint-Gillis,Ozfair,Jean Volderslaan,['Eethuis'],1060,Jean Volderslaan 1060 Sint-Gillis,"(50.831634943997585, 4.3436130368112025)",False
544,Brugge,Passage,Dweersstraat 26,['Eethuis'],8000,Dweersstraat 26 8000 Brugge,"(51.20586663915748, 3.2191247459149515)",False
546,Gent,Pasta Presto,Sint-Amandstraat 17,['Eethuis'],9000,Sint-Amandstraat 17 9000 Gent,"(51.04332279167811, 3.7249256805045556)",False
556,Gent,Petrus,Sint-Amandstraat 15,"['Eethuis', 'Snack', 'Veganvriendelijk']",9000,Sint-Amandstraat 15 9000 Gent,"(51.04351139722262, 3.7248945699168647)",False
563,Hasselt,Pocomatto,Dorpsstraat 42,['Eethuis'],3500,Dorpsstraat 42 3500 Hasselt,"(50.932379914450635, 5.335700399626532)",False


In [25]:
df_notfound[70:80]

,city,name,street,tags,zipcode,full_address,latlon,missing
566,Zegelsem (Brakel),Ponderspijs,"""Rovorst 33 9660 Zegelsem (Brakel)""",['Cateraar'],9660,"""Rovorst 33 9660 Zegelsem (Brakel)"" 9660 Zegel...",NaN,True
577,Aalst,Qarfa,Stationstraat 13,[],9300,Stationstraat 13 9300 Aalst,NaN,True
586,Kortrijk,Repos et Repas,Bouwcentrum Pottelberg Engelse Wandeling 2 K17,['Eethuis'],8500,Bouwcentrum Pottelberg Engelse Wandeling 2 K17...,NaN,True
589,Brussel,Ricotta & Parmesan,Schildknaapstraat 31,['Eethuis'],1000,Schildknaapstraat 31 1000 Brussel,NaN,True
597,Gent,S.M.A.K café by Grade,Citadelpark,['Snack'],9000,Citadelpark 9000 Gent,NaN,True
608,Kasterlee,Seven Hill,Geelsebaan 29/2,['Eethuis'],9460,Geelsebaan 29/2 9460 Kasterlee,NaN,True
610,Vorst,Sinstreetfood,Koningsstraat 165,"['Snack', '100% vegetarisch', 'EVA voordeel']",1190,Koningsstraat 165 1190 Vorst,NaN,True
612,Brussel,Skievelat,J. Stevensstraat 16-18,['Eethuis'],1000,J. Stevensstraat 16-18 1000 Brussel,NaN,True
616,Gent,Soeperstar,Makelaarstraat 24,['Cateraar'],9000,Makelaarstraat 24 9000 Gent,NaN,True
646,Waardamme,t Oud Gemeentehuis,Kortrijksstraat 405,['Eethuis'],8020,Kortrijksstraat 405 8020 Waardamme,NaN,True


In [28]:
overwriteAddress('Rovorst 33 9660 Zegelsem',566,df_notfound)
overwriteAddress('Stationsstraat 13 9300 Aalst',577,df_notfound)
overwriteAddress('Engelse Wandeling 2 8500 Kortrijk',586,df_notfound)
overwriteAddress('Schildknaaps Straat 31 1000 Brussel',589,df_notfound)
overwriteAddress('Jan Hoetplein 1 9000 Gent',597,df_notfound)

overwriteAddress('Geelsebaan 29/2 2460 Kasterlee',608,df_notfound)
overwriteAddress('Rue Royale 165 1190 Vorst',610,df_notfound)
overwriteAddress('Joseph Stevensstraat 16-18 1000 Brussel',612,df_notfound)
overwriteAddress('Makelaarsstraat 24 9000 Gent',616,df_notfound)
overwriteAddress('Kortrijksestraat 405 8020 Waardamme',646,df_notfound)

df_notfound[70:80]

,city,name,street,tags,zipcode,full_address,latlon,missing
566,Zegelsem,Ponderspijs,Rovorst 33,['Cateraar'],9660,Rovorst 33 9660 Zegelsem,"(50.79373791875597, 3.7175001627977577)",False
577,Aalst,Qarfa,Stationsstraat 13,[],9300,Stationsstraat 13 9300 Aalst,"(50.941481263289425, 4.037367638854749)",False
586,Kortrijk,Repos et Repas,Engelse Wandeling 2,['Eethuis'],8500,Engelse Wandeling 2 8500 Kortrijk,"(50.81461894247178, 3.245370989893236)",False
589,Brussel,Ricotta & Parmesan,Schildknaaps Straat 31,['Eethuis'],1000,Schildknaaps Straat 31 1000 Brussel,"(50.84904941738051, 4.354438613128288)",False
597,Gent,S.M.A.K café by Grade,Jan Hoetplein 1,['Snack'],9000,Jan Hoetplein 1 9000 Gent,"(51.03817709725133, 3.722586781062788)",False
608,Kasterlee,Seven Hill,Geelsebaan 29/2,['Eethuis'],2460,Geelsebaan 29/2 2460 Kasterlee,"(51.23387387414675, 4.971535699587303)",False
610,Vorst,Sinstreetfood,Rue Royale 165,"['Snack', '100% vegetarisch', 'EVA voordeel']",1190,Rue Royale 165 1190 Vorst,"(50.8125173892532, 4.324027938080416)",False
612,Brussel,Skievelat,Joseph Stevensstraat 16-18,['Eethuis'],1000,Joseph Stevensstraat 16-18 1000 Brussel,"(50.841462059756566, 4.352765477053187)",False
616,Gent,Soeperstar,Makelaarsstraat 24,['Cateraar'],9000,Makelaarsstraat 24 9000 Gent,"(51.071784101804795, 3.7301275120078174)",False
646,Waardamme,t Oud Gemeentehuis,Kortrijksestraat 405,['Eethuis'],8020,Kortrijksestraat 405 8020 Waardamme,"(51.109887151544775, 3.221312856732368)",False


In [29]:
df_notfound[80:95]

,city,name,street,tags,zipcode,full_address,latlon,missing
653,Elsene,Tan,Waterleidingstraat 95,['Eethuis'],1050,Waterleidingstraat 95 1050 Elsene,NaN,True
655,Brussel,Tarte Julie,E. Jacqmainlaan 56,"['Eethuis', 'EVA voordeel']",1000,E. Jacqmainlaan 56 1000 Brussel,NaN,True
663,Rocourt,Tasty III,Rue de la casquette 17,"['Snack', '100% vegetarisch', 'EVA voordeel']",4000,Rue de la casquette 17 4000 Rocourt,NaN,True
670,Leuven,Thai House,Alfonssmetsplein,['Eethuis'],3000,Alfonssmetsplein 3000 Leuven,NaN,True
687,Elsene,Tom Yam,Chaussee de Boondaalsesteenweg 341,['Eethuis'],1050,Chaussee de Boondaalsesteenweg 341 1050 Elsene,NaN,True
688,Brussel,Toukoul,Lakenstraat 34,"['Eethuis', 'EVA voordeel', 'Veganvriendelijk']",1000,Lakenstraat 34 1000 Brussel,NaN,True
692,Gent,Tuin van Eten,Kortrijksestesteenweg 573,"['Eethuis', 'Veganvriendelijk']",9000,Kortrijksestesteenweg 573 9000 Gent,NaN,True
705,vegetarisch,Veggiebites,Cateraar,[],100%,Cateraar 100% vegetarisch,NaN,True
710,Merelbeke,Villa Florazicht,Hundelgemsesteenweg 182,['Eethuis'],9082,Hundelgemsesteenweg 182 9082 Merelbeke,NaN,True
721,Gent,Wereldrestaurant De Centrale,Kraankinderstraat 2,['Eethuis'],9000,Kraankinderstraat 2 9000 Gent,NaN,True


In [30]:
overwriteAddress('Waterleidingsstraat 95 1050 Elsene',653,df_notfound)
overwriteAddress('Emile Jacqmainlaan 56 1000 Brussel',655,df_notfound)

#print(latlon_from_address('')) #wallonie => 50°38'33.6"N 5°34'09.1"E
df_notfound.set_value(663, 'latlon', (50.642521, 5.569165))
df_notfound.set_value(663, 'missing', False)
                          
overwriteAddress('Alfons Smetsplein 3000 Leuven',670,df_notfound)
overwriteAddress('Boondaalse Steenweg 341 1050 Elsene',687,df_notfound)

overwriteAddress('Lakensestraat 34 1000 Brussel',688,df_notfound)
overwriteAddress('Kortrijksesteenweg 573 9000 Gent',692,df_notfound)
#print(latlon_from_address('')) #BAD Record
overwriteAddress('Hundelgemsesteenweg 182 9820 Merelbeke',710,df_notfound)
overwriteAddress('Kraankindersstraat 2 9000 Gent',721,df_notfound)

overwriteAddress('Pelgrimstraat 2 2000 Antwerpen',733,df_notfound)

In [31]:
df_notfound[80:95]

,city,name,street,tags,zipcode,full_address,latlon,missing
653,Elsene,Tan,Waterleidingsstraat 95,['Eethuis'],1050,Waterleidingsstraat 95 1050 Elsene,"(50.82422075201777, 4.358930651079416)",False
655,Brussel,Tarte Julie,Emile Jacqmainlaan 56,"['Eethuis', 'EVA voordeel']",1000,Emile Jacqmainlaan 56 1000 Brussel,"(50.85347214426758, 4.353542644100108)",False
663,Rocourt,Tasty III,Rue de la casquette 17,"['Snack', '100% vegetarisch', 'EVA voordeel']",4000,Rue de la casquette 17 4000 Rocourt,"(50.642521, 5.569165)",False
670,Leuven,Thai House,Alfons Smetsplein,['Eethuis'],3000,Alfons Smetsplein 3000 Leuven,"(50.87734615184162, 4.703494168185616)",False
687,Elsene,Tom Yam,Boondaalse Steenweg 341,['Eethuis'],1050,Boondaalse Steenweg 341 1050 Elsene,"(50.81860156448049, 4.383822497652958)",False
688,Brussel,Toukoul,Lakensestraat 34,"['Eethuis', 'EVA voordeel', 'Veganvriendelijk']",1000,Lakensestraat 34 1000 Brussel,"(50.85231214252864, 4.350901610751696)",False
692,Gent,Tuin van Eten,Kortrijksesteenweg 573,"['Eethuis', 'Veganvriendelijk']",9000,Kortrijksesteenweg 573 9000 Gent,"(51.035159186476875, 3.7165121218884276)",False
705,vegetarisch,Veggiebites,Cateraar,[],100%,Cateraar 100% vegetarisch,NaN,True
710,Merelbeke,Villa Florazicht,Hundelgemsesteenweg 182,['Eethuis'],9820,Hundelgemsesteenweg 182 9820 Merelbeke,"(51.01910438947989, 3.7533206846707556)",False
721,Gent,Wereldrestaurant De Centrale,Kraankindersstraat 2,['Eethuis'],9000,Kraankindersstraat 2 9000 Gent,"(51.061208593462986, 3.734351476392869)",False


# Reassemble data frame

In [37]:
df_found_final = all_data[~all_data['missing']]
df_found_final.shape

(644, 8)

In [38]:
df_notfound_final = df_notfound[~df_notfound['missing']]
df_notfound_final.shape

(84, 8)

In [41]:
df_all_final = pd.concat([df_found_final, df_notfound_final])

In [42]:
df_all_final.shape

(728, 8)

In [43]:
df_all_final.to_csv('EVA_restoswithlocationsANDfixes.csv', index=False)